In [42]:
import os
import dotenv
import google.generativeai as genai

dotenv.load_dotenv()
    
def generate_output(system_prompt, user_prompt):
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        system_instruction=system_prompt
    )

    response = model.generate_content(
        user_prompt,
        generation_config={
            "temperature": 0
        }
    )

    return response.text

In [43]:
import re
import time
from tqdm import tqdm
def replace_multi_table(text, replace_text):
    """find the first pattern and replace"""
    table_pattern = r'(\|.*\|(?:\n\|.*\|)+)'
    text = re.sub(table_pattern, replace_text, text, count=1)
        
    return text

def extract_tables_from_corpus(text):
    """extract table from corpus based on pattern"""
    table_pattern = r'(\|.*\|(?:\n\|.*\|)+)'
    table = re.findall(table_pattern, text)
    return table

In [ ]:
import time
import re
import json
from typing import List, Dict

def robust_json_parser(summary_json_string: str) -> List[Dict]:
    """
    Cleans a raw string output from the model, splits it by dictionary delimiters (},{),
    and attempts to parse each dictionary individually before collecting the final list.
    
    Args:
        summary_json_string: The raw string output from the Gemini model.
        
    Returns:
        A list of successfully parsed dictionaries containing '_id' and 'summary'.
    """
    
    # 1. Initial Cleaning (Remove Markdown Fences and surrounding spaces)
    # This addresses the common '```json' error
    cleaned = summary_json_string.strip()
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].strip()
    if cleaned.endswith("```"):
        cleaned = cleaned.rstrip("`").strip()

    # 2. Extract the inner content (the list items)
    # Remove the outer brackets [ and ] to get the raw string of dictionaries
    if cleaned.startswith('[') and cleaned.endswith(']'):
        inner_content = cleaned[1:-1].strip()
    else:
        # If brackets are missing, assume the content is just the dictionaries
        inner_content = cleaned
    
    # Check if we have anything left to process
    if not inner_content:
        return []

    # 3. Split by the delimiter "},{"
    # This separates the string into individual dictionary strings
    # The delimiter itself is not included in the split segments
    dict_strings = inner_content.split('},{')
    
    final_summaries = []
    
    for i, dict_str in enumerate(dict_strings):
        # 4. Reconstruct the full dictionary string
        # The split removed the necessary { and } at the beginning/end of the dictionary string
        if dict_str.startswith('{'):
            dict_str = dict_str[1:]
        if dict_str.endswith('}'):
            dict_str = dict_str[:-1]
        
        reconstructed_dict = "{" + dict_str + "}"
        
        # We need to fix the first and last segments:
        if i == 0 and not reconstructed_dict.startswith('{'):
            reconstructed_dict = '{' + reconstructed_dict.lstrip('{')
        if i == len(dict_strings) - 1 and not reconstructed_dict.endswith('}'):
            reconstructed_dict = reconstructed_dict.rstrip('}') + '}'
        
        # 5. Final attempt to parse the individual dictionary
        try:
            summary_dict = json.loads(reconstructed_dict)
            
            # 6. Validate essential keys before adding
            if '_id' in summary_dict and 'summary' in summary_dict:
                final_summaries.append(summary_dict)
            else:
                print(f"Warning: Dictionary {i+1} missing '_id' or 'summary' keys. Skipping.")
                
        except json.JSONDecodeError as e:
            # If a single dictionary fails to parse, we log the error and move to the next one
            print(f"Warning: Failed to parse dictionary segment {i+1}. Error: {e}")
            print(f"Failed string: {reconstructed_dict}")
            continue
    print("This is our final summaries length:", len(final_summaries), "/", len(dict_strings))
    print(final_summaries[:2])
    return final_summaries

def batch_summarize(system_prompt, texts: list[dict], batch_size=100):
    """takes the whole list of dict with tables in markdown format as string and parse them into multiple batches for Gemini summary.

    Args:
        system_prompt (str): _description_
        texts (list): _description_
        batch_size (int, optional): _description_. Defaults to 100.
    """
    summaries = []
    total_calls = (len(texts) + batch_size - 1) // batch_size
    for i in tqdm(range(total_calls)):
        batch_texts = texts[i*batch_size:(i+1)*batch_size]
        user_prompt = f"{batch_texts}"
        summary_json_string = generate_output(system_prompt, user_prompt)        
        try:
            summary_list = robust_json_parser(summary_json_string)            
            # Use .extend() to add elements of one list to another.
            summaries.extend(summary_list)
            
        except json.JSONDecodeError as e:
            # Handle cases where the model output is malformed and not valid JSON
            print(f"\n[ERROR] Failed to parse JSON output for batch {i}: {e}")
            print(f"Malformed output: {summary_json_string[:200]}...") # Print beginning of bad output
            # You might choose to log the bad data or skip the batch
        time.sleep(1.2)  # To avoid hitting rate limits
    return summaries
    

In [45]:
from utils.json_csv_utils import load_jsonl
from utils.multihiertt_table_extraction import MultiHierttTableExtractor

instruction = """
Retrieve all facts in the following table as natural language descriptions, without introductory phrases. 

# Instruction:
You will receive a list of dict that contain tables in string format, summarize each table content and return the result in the provided sample format exactly as a string.

# Input format:
"
[
  {
    "_id": "table_1_id",
    "table_content": "<table_content_1>"
  },
  {
    "_id": "table_2_id",
    "table_content": "<table_content_2>"
  }
]
"

# Output format: (DO NOT INCLUDE THESE QUOTES IN THE MODEL'S RESPONSE)
"
  {
      "_id": "table_1_id",
      "summary": "<summary_of_table_1>"
  },
  {
      "_id": "table_2_id",
      "summary": "<summary_of_table_2>"
  }
"

"""
# df = pd.read_csv("./financerag/dataset_csv/MultiHiertt_corpus.csv")
tasks = [
  # "ConvFinQA",
  # "FinanceBench",
  # "FinQA",
  "MultiHiertt",
  # "FinQABench",
  # "FinDER",
  # "TATQA"
]

call_count = 0
# extracted tables is stored as a json

for task in tasks:
  extracted_tables = []
  extracted_corpus = load_jsonl(f"./dataset/{task}/corpus.jsonl")
  
  for item in tqdm(extracted_corpus):
      text = item["text"]

      if not isinstance(text, str):
          continue

      tables = extract_tables_from_corpus(text)
      if len(tables) > 0:
          for i in range(len(tables)):
              extracted_tables.append({
                  "_id": item['_id'],
                  "table_content": tables[i]
              })
              
  summary = batch_summarize(instruction, extracted_tables, batch_size=100)
  print(summary[:3])
  if len(summary) > 0:
    with open(f"./dataset/{task}/MultiHiertt_extracted_tables_summaries.jsonl", "w") as f:
      for entry in summary:
        f.write(json.dumps(entry) + "\n")
              
  for item in summary:
    item_id = item["_id"]
    summary_text = item["summary"]
    for corpus_item in extracted_corpus:
        if corpus_item["_id"] == item_id:
            original_text = corpus_item["text"]
            new_text = replace_multi_table(original_text, summary_text)
            corpus_item["text"] = new_text
            break
  with open(f"./dataset/{task}/corpus_with_summarized_tables.jsonl", "w") as f:
    for entry in extracted_corpus:
      f.write(json.dumps(entry) + "\n")
  # extracted_table = extract_tables_from_corpus(extracted_corpus)

# for idx in range(len(df)):
#     text = df.at[idx, "text"]

#     if not isinstance(text, str):
#         continue

#     if MultiHierttTableExtractor.has_table(text):
#         result = MultiHierttTableExtractor.extract_table(text)
#         if result:
#             _, table_content = result
#             parsed = MultiHierttTableExtractor.parse_table_to_text(table_content)
#             replaced = generate_output(instruction, parsed)
#             new_text = replace_multi_table(text, replaced)

#             # update only this row
#             df.at[idx, "text"] = new_text

#             call_count += 1
#             if call_count % 1000 == 0:
#                 time.sleep(60)

# write final file
# df.to_csv("./financerag/dataset_csv/MultiHiertt_corpus_cleaned.csv", index=False)


Loading data from ./dataset/MultiHiertt/corpus.jsonl
Loaded 10475 items


  0%|          | 0/80 [00:00<?, ?it/s]

Failed string: {{
      "_id": "d8e4ea4ac",
      "summary": "Investment return for the years ended December 31, 2006 was $192 million. Investment return for the years ended December 31, 2005 was $-26 million. Expense for the years ended December 31, 2006 was $45 million. Expense for the years ended December 31, 2005 was $11 million. In-force/Persistency for the years ended December 31, 2006 was $-7 million. In-force/Persistency for the years ended December 31, 2005 was $-33 million. Policyholder dividends and other for the years ended December 31, 2006 was $-39 million. Policyholder dividends and other for the years ended December 31, 2005 was $-11 million. Total for the years ended December 31, 2006 was $191 million. Total for the years ended December 31, 2005 was $-59 million."
  },
  {
      "_id": "d8d151e68",
      "summary": "The total cumulative return for Edwards Lifesciences in 2012 was $127.54. The total cumulative return for Edwards Lifesciences in 2013 was $93.01. The tota

  1%|▏         | 1/80 [04:29<5:54:20, 269.11s/it]

Failed string: {{
    "_id": "d8c3f3b86",
    "summary": "The risk-free interest rate was 0.35% in 2013, 0.16% in 2012, and 1.35% in 2011. The expected dividend yield was 0% for all three years. The expected volatility of ANSYS Stock Price was 25% in 2013, 28% in 2012, and 40% in 2011. The expected volatility of the NASDAQ Composite Index was 20% in 2013 and 2012, and 25% in 2011. The expected term was 2.8 years in 2013 and 2012, and 2.9 years in 2011. The correlation factor was 0.70 in 2013, 0.75 in 2012, and 0.70 in 2011."
  },
  {
    "_id": "d8e0599ba",
    "summary": "At the beginning of the year, there were 7,328 shares with an average grant date fair value of $49.17 per share. 2,134 shares were granted with an average grant date fair value of $51.91 per share. 4,372 shares were earned/vested with an average grant date fair value of $49.14 per share. 91 shares were canceled with an average grant date fair value of $51.18 per share. At the end of the year, there were 4,999 shares 

  2%|▎         | 2/80 [08:21<5:26:06, 250.85s/it]


KeyboardInterrupt: 

In [ ]:
from utils.json_csv_utils import load_jsonl

read_result = load_jsonl("./dataset/FinQABench/corpus.jsonl")
read_result[:3]

Loading data from ./dataset/FinQABench/corpus.jsonl
Loaded 92 items
Apple Inc.
CONSOLIDATED STATEMENTS OF OPERATIONS
(In millions, except number of shares which are reflected in thousands and per share amounts)
Years ended
September 24,
2022September 25,
2021September 26,
2020
Net sales:
   Products $ 316,199 $ 297,392 $ 220,747 
   Services  78,129  68,425  53,768 
Total net sales  394,328  365,817  274,515 
Cost of sales:
   Products  201,471  192,266  151,286 
   Services  22,075  20,715  18,273 
Total cost of sales  223,546  212,981  169,559 
Gross margin  170,782  152,836  104,956 
Operating expenses:
Research and development  26,251  21,914  18,752 
Selling, general and administrative  25,094  21,973  19,916 
Total operating expenses  51,345  43,887  38,668 
Operating income  119,437  108,949  66,288 
Other income/(expense), net  (334)  258  803 
Income before provision for income taxes  119,103  109,207  67,091 
Provision for income taxes  19,300  14,527  9,680 
Net income $ 99,